In [2]:
pip install numpy scipy pandas yfinance


Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.stats import norm
import math

# Black-Scholes formula for European call option
def black_scholes_call(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call_price

# Load Nifty50 data (for Spot Price, historical data can be pulled from Yahoo Finance)
def get_nifty50_spot_price():
    nifty50 = yf.Ticker("^NSEI")
    hist = nifty50.history(period="1d")
    spot_price = hist['Close'].iloc[-1]
    return spot_price

# Example function to calculate accuracy between model and real option prices
def compare_option_prices(nifty_spot_price, real_option_data):
    calculated_prices = []
    errors = []

    for index, row in real_option_data.iterrows():
        S = nifty_spot_price          # Spot Price
        K = row['strike']             # Strike Price
        T = row['time_to_expiry']     # Time to Expiry (in years)
        r = row['risk_free_rate']     # Risk-Free Interest Rate
        sigma = row['implied_volatility']  # Implied Volatility

        # Calculate Black-Scholes call option price
        calculated_call_price = black_scholes_call(S, K, T, r, sigma)
        calculated_prices.append(calculated_call_price)
        
        # Compare with real option price and calculate error
        real_call_price = row['real_call_price']
        error = abs(calculated_call_price - real_call_price)
        errors.append(error)

    real_option_data['calculated_call_price'] = calculated_prices
    real_option_data['error'] = errors
    return real_option_data

if __name__ == "__main__":
    # Example Nifty50 spot price (fetch it live)
    nifty_spot_price = get_nifty50_spot_price()
    
    # Columns: strike, time_to_expiry (in years), risk_free_rate, implied_volatility, real_call_price
    real_option_data = pd.DataFrame({
        'strike': [17500, 17600, 17700],
        'time_to_expiry': [0.1, 0.1, 0.1],  # 18 days to expiry (18/365)
        'risk_free_rate': [0.065, 0.065, 0.065],  # risk-free rate (6.5%)
        'implied_volatility': [0.12, 0.14, 0.15],  # implied volatilities
        'real_call_price': [7200, 7000, 6800]  # real option prices (from market data)
    })
    
    # Compare calculated and real option prices
    result = compare_option_prices(nifty_spot_price, real_option_data)
    
    # Display the results
    print(result)


   strike  time_to_expiry  risk_free_rate  implied_volatility  \
0   17500             0.1           0.065                0.12   
1   17600             0.1           0.065                0.14   
2   17700             0.1           0.065                0.15   

   real_call_price  calculated_call_price       error  
0             7200            7186.031503   13.968497  
1             7000            7086.679395   86.679395  
2             6800            6987.327287  187.327287  


In [1]:

# Monte Carlo simulation for European Call Option Pricing
def monte_carlo_call_option(S, K, T, r, sigma, n_simulations=10000):
    np.random.seed(42)  # For reproducibility
    payoff_sum = 0

    # Simulate asset price paths
    for _ in range(n_simulations):
        # Simulate the percentage change in price based on geometric Brownian motion
        Z = np.random.normal(0, 1)  # Draw a random sample from standard normal distribution
        S_T = S * np.exp((r - 0.5 * sigma ** 2) * T + sigma * np.sqrt(T) * Z)
        payoff = max(S_T - K, 0)  # Call option payoff
        payoff_sum += payoff

    # Calculate the expected payoff and discount it to the present value
    call_price = (payoff_sum / n_simulations) * np.exp(-r * T)
    return call_price

# Load Nifty50 data (for Spot Price, historical data can be pulled from Yahoo Finance)
def get_nifty50_spot_price():
    nifty50 = yf.Ticker("^NSEI")
    hist = nifty50.history(period="1d")
    spot_price = hist['Close'].iloc[-1]
    return spot_price

# Example function to calculate accuracy between model and real option prices
def compare_option_prices_monte_carlo(nifty_spot_price, real_option_data, n_simulations=10000):
    calculated_prices = []
    errors = []

    for index, row in real_option_data.iterrows():
        S = nifty_spot_price          # Spot Price
        K = row['strike']             # Strike Price
        T = row['time_to_expiry']     # Time to Expiry (in years)
        r = row['risk_free_rate']     # Risk-Free Interest Rate
        sigma = row['implied_volatility']  # Implied Volatility

        # Calculate option price using Monte Carlo simulation
        calculated_call_price = monte_carlo_call_option(S, K, T, r, sigma, n_simulations)
        calculated_prices.append(calculated_call_price)
        
        # Compare with real option price and calculate error
        real_call_price = row['real_call_price']
        error = abs(calculated_call_price - real_call_price)
        errors.append(error)

    real_option_data['calculated_call_price_mc'] = calculated_prices
    real_option_data['error_mc'] = errors
    return real_option_data

if __name__ == "__main__":
    # Example Nifty50 spot price (fetch it live)
    nifty_spot_price = get_nifty50_spot_price()
    
   
    # Columns: strike, time_to_expiry (in years), risk_free_rate, implied_volatility, real_call_price
    real_option_data = pd.DataFrame({
        'strike': [17500, 17600, 17700],
        'time_to_expiry': [0.05, 0.05, 0.05],  # Assuming 18 days to expiry (18/365)
        'risk_free_rate': [0.065, 0.065, 0.065],  # risk-free rate (6.5%)
        'implied_volatility': [0.12, 0.14, 0.15],  # implied volatilities
        'real_call_price': [7200, 7000, 6800]  # Example real option prices (from market data)
    })
    
    # Compare calculated and real option prices using Monte Carlo simulation
    result = compare_option_prices_monte_carlo(nifty_spot_price, real_option_data)
    
    # Display the results
    print(result)


   strike  time_to_expiry  risk_free_rate  implied_volatility  \
0   17500            0.05           0.065                0.12   
1   17600            0.05           0.065                0.14   
2   17700            0.05           0.065                0.15   

   real_call_price  calculated_call_price_mc    error_mc  
0             7200               7128.085382   71.914618  
1             7000               7028.197081   28.197081  
2             6800               6928.416440  128.416440  
